In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.text, 'html.parser')

In [57]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

In [58]:
new_df=df[0]

In [59]:
new_df.columns = new_df.iloc[0]


In [61]:
df = new_df.drop(new_df.index[0])
df.head()


,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


# Ignore rows which are having Borough as "Not Assigned"

In [66]:
df =df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


# Combine duplicate Neighbourhood into single rows

In [73]:
df=df.groupby([df.Postcode,df.Borough],as_index=False).agg(', '.join)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [86]:
df.loc[df.Neighbourhood=='Not assigned']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned
